In [ ]:
# Este modelo funciona con una version anterior de openai... para lo cual en consola primiero se debe desinstalar la actual: con "pip uninstall openai" para luego instalar "pip install openai==0.27.8"

# Luego instalar las librerías necesarias

! pip install openai azure-search-documents python-dotenv transformers scikit-learn matplotlib seaborn numpy spacy
! python -m spacy download es_core_news_sm

import os
import openai
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv, find_dotenv
from transformers import pipeline
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

# Cargar archivo .env
dotenv_path = find_dotenv()
if dotenv_path:
    print(f"Archivo .env encontrado en: {dotenv_path}")
    load_dotenv(dotenv_path)
else:
    print("Archivo .env no encontrado. Asegúrate de que exista y esté en la ruta correcta.")

# Variables necesarias para Azure Cognitive Search
AZURE_SEARCH_ENDPOINT = os.getenv("SEARCH_SERVICE_ENDPOINT", "").strip()
AZURE_SEARCH_KEY = os.getenv("SEARCH_API_KEY", "").strip()
INDEX_NAME = os.getenv("INDEX_NAME", "").strip()

# Variables necesarias para OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "").strip()  # Línea que obtiene la clave de OpenAI
MODEL_NAME = os.getenv("MODEL_NAME", "text-embedding-ada-002").strip()

# Configurar cliente OpenAI
openai.api_key = OPENAI_API_KEY  # Línea que configura el cliente de OpenAI

# Configuración para Azure OpenAI (opcional)
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "").strip()
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY", "").strip()
ENGINE_NAME = os.getenv("ENGINE_NAME", MODEL_NAME)  # Usa el engine o modelo según configuración

# Para usar Azure OpenAI, configura el cliente así:
if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_KEY:
    openai.api_type = "azure"
    openai.api_base = AZURE_OPENAI_ENDPOINT
    openai.api_version = "2023-03-15-preview"
    openai.api_key = AZURE_OPENAI_KEY

# Cliente de Azure Cognitive Search
search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

# Carga pipeline de factualidad en español
factual_pipeline = pipeline(
    "question-answering",
    model="bertin-project/bertin-roberta-base-spanish",
    tokenizer="bertin-project/bertin-roberta-base-spanish",
    framework="pt"
)

# Función para obtener embeddings de OpenAI
#Reduciendo la frecuencia de llamadas (Rate-Limiting con Retrasos)para tener mas cuota... 
import time
import openai

def obtener_embeddings_openai_con_reintento(texto, max_reintentos=5, tiempo_espera=60):
    """Obtiene embeddings con reintentos en caso de alcanzar el límite de tasa."""
    for intento in range(max_reintentos):
        try:
            response = openai.Embedding.create(
                input=texto,
                engine=ENGINE_NAME
            )
            return response['data'][0]['embedding']
        except openai.error.RateLimitError:
            print(f"Rate limit alcanzado. Esperando {tiempo_espera} segundos... (Intento {intento + 1}/{max_reintentos})")
            time.sleep(tiempo_espera)
    raise Exception("No se pudo obtener embeddings después de varios intentos.")


# def obtener_embeddings_openai(texto):
#     """Obtiene los embeddings de Azure OpenAI para un texto dado."""
#     # Usa ENGINE_NAME en lugar de MODEL_NAME si estás trabajando con Azure OpenAI
#     engine_or_model = os.getenv("ENGINE_NAME", MODEL_NAME)
    
#     response = openai.Embedding.create(
#         input=texto,
#         engine=engine_or_model  # Cambia 'model' por 'engine' para Azure OpenAI
#     )
#     return response['data'][0]['embedding']


# Función para verificar factualidad
def verificar_factualidad_respuesta(respuesta, documentos_referencia, umbral=0.6):
    """Verifica la factualidad combinando pipeline y similitud semántica utilizando embeddings de OpenAI."""
    scores_factualidad = []
    scores_similitud = []

    for doc in documentos_referencia:
        # Factualidad (QA Pipeline)
        resultado = factual_pipeline(question=respuesta, context=doc["content"])
        scores_factualidad.append(resultado["score"])

        # Similitud semántica usando embeddings de OpenAI
        emb_respuesta = obtener_embeddings_openai(respuesta)
        emb_contexto = obtener_embeddings_openai(doc["content"])
        similarity = cosine_similarity([emb_respuesta], [emb_contexto])[0][0]
        scores_similitud.append(similarity)

    # Promedios
    avg_factualidad = np.mean(scores_factualidad) if scores_factualidad else 0
    avg_similitud = np.mean(scores_similitud) if scores_similitud else 0

    # Pesos ajustados
    peso_factualidad = 0.4
    peso_similitud = 0.6
    puntaje_total = (avg_factualidad * peso_factualidad) + (avg_similitud * peso_similitud)

    # Resultado final
    es_factual = puntaje_total >= umbral
    return puntaje_total, es_factual

# Evaluar respuesta
def evaluar_respuesta(respuesta):
    """Evalúa si la respuesta es factual."""
    documentos = buscar_en_azure(respuesta)
    if not documentos:
        print("Advertencia: No se encontraron documentos. Evaluación como alucinación.")
        return 1  # Asumir alucinación

    _, es_factual = verificar_factualidad_respuesta(respuesta, documentos)
    return 0 if es_factual else 1

# Función para buscar en Azure
def buscar_en_azure(query):
    """Busca información relevante en Azure Search."""
    resultados = search_client.search(query)
    documentos = []
    for r in resultados:
        documentos.append({
            "content": r.get("content", ""),  
            "source": r.get("@search.documentkey", "unknown_source") 
        })
    return documentos

# Generar datos para matriz de confusión
def generar_datos_para_matriz(respuestas_y_groundtruth):
    """Genera predicciones y etiquetas reales."""
    resultados = []
    for entrada in respuestas_y_groundtruth:
        respuesta = entrada["respuesta"]
        groundtruth = entrada["groundtruth"]
        prediccion = evaluar_respuesta(respuesta)
        resultados.append({"groundtruth": groundtruth, "prediction": prediccion})
    return resultados

# Calcular y mostrar matriz de confusión
def calcular_matriz_confusion(resultados):
    """Calcula y visualiza la matriz de confusión."""
    groundtruth = [r["groundtruth"] for r in resultados]
    prediction = [r["prediction"] for r in resultados]

    cm = confusion_matrix(groundtruth, prediction)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=["Factual", "Alucinación"], yticklabels=["Factual", "Alucinación"])
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta Real')
    plt.title('Matriz de Confusión')
    plt.show()
    print(classification_report(groundtruth, prediction, target_names=["Factual", "Alucinación"]))

# Flujo principal
if __name__ == "__main__":
    respuestas_y_groundtruth = [
        {"respuesta": "La huella de carbono mide emisiones de CO₂ producidas por actividades humanas.", "groundtruth": 0},
    ]

    resultados = generar_datos_para_matriz(respuestas_y_groundtruth)
    calcular_matriz_confusion(resultados)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
zsh:1: command not found: python
Archivo .env encontrado en: /Users/luisalbertocerelli/Desktop/00-Todo/14_Hackaton_Real/PRINCIPAL/02_Mejoras_al_modelo/codigo_real/.env


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0


RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.

In [ ]:
import openai
import os

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

# Probar el engine
engine_name = os.getenv("ENGINE_NAME")
try:
    response = openai.Embedding.create(
        input="Hola, mundo!",
        engine=engine_name
    )
    print("El motor funciona correctamente:", response)
except Exception as e:
    print("Error con el motor:", e)
